<a href="https://colab.research.google.com/github/vinodhiniasha-eng/255-Data-Mining/blob/main/HW1_Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 1 Part 3: Preprocessing the BayWheels Bike Dataset

The original dataset is available here: https://s3.amazonaws.com/baywheels-data/index.html
You will each work on a subset of the dataset as follows:

Take the last two digits of your SJSU ID. Pick the month corresponding to the last digit of your SJSU ID. For example, if your SJSU ID ends in 7, you will pick July. If the digit in the tens place is even, pick the dataset for the year 2020. If it is odd, pick the dataset for the year 2021.

For example, if your SJSU ID is 123456789, the last digit of your SJSU ID is 9 and the digit in the tens place is even. So, you will pick the dataset for September 2020, i.e., 202009-baywheels-tripdata.csv.zip. If your SJSU ID is 098765432, the last digit of your SJSU ID is 2 and the digit in the tens place is odd. So, you will pick the dataset for February 2021, i.e., 202102-baywheels-tripdata.csv.zip.

Perform the following tasks on the provided dataset. High pass and low pass specifications are on Canvas. Please note that ALL questions labelled [LP] must be completed satisfactorily to receive a low pass on the assignment. The [HP] question specifications combine all parts of the homework. Please refer to Canvas for more details.

Note that if you are unable to complete any of the LP questions satisfactorily, you will receive a grade of "revision required". You can revise and resubmit your work in exchange for a token. Please review the syllabus for more information on specifications grading.

**VERY IMPORTANT**: Include **ALL** the references you used for this assignment, including names of classmates you discuss with. Failure to cite your sources counts as an act of academic dishonesty and will be taken seriously without zero tolerance. You will automatically receive a “fail” grade in the homework and further serious penalties may be imposed.

NOTE: You can look for help on the Internet but refrain from referencing too much. Please cite all your sources in your submission.
When you submit your assignment, you automatically agree to the following statement. If you do not agree, it is your responsibility to provide the reason.

“*I affirm that I have neither given nor received unauthorized help in completing this homework. I am not aware of others receiving such help. I have cited all the sources in the solution file.*”

In [1]:
## Your code goes here. Import the csv into a pandas dataframe here
import pandas as pd
import zipfile

zip_path = "/content/202101-baywheels-tripdata.csv.zip"

# Read the single CSV inside the zip
with zipfile.ZipFile(zip_path, "r") as z:
    csv_name = [n for n in z.namelist() if n.endswith(".csv")][0]
    with z.open(csv_name) as f:
        df = pd.read_csv(f)

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,357CDE244D24405B,electric_bike,2021-01-26 11:32:59,2021-01-26 11:38:21,NaN,NaN,NaN,NaN,37.76,-122.41,37.76,-122.42,casual
1,19A3E1F4211D0EE8,electric_bike,2021-01-26 14:16:37,2021-01-26 14:19:24,NaN,NaN,NaN,NaN,37.77,-122.41,37.76,-122.41,casual
2,27004D90ADC81AFF,electric_bike,2021-01-26 14:02:37,2021-01-26 14:06:35,NaN,NaN,NaN,NaN,37.76,-122.42,37.77,-122.41,casual
3,2F81FCA3D9CD056A,electric_bike,2021-01-26 15:03:05,2021-01-26 15:07:25,NaN,NaN,NaN,NaN,37.76,-122.41,37.76,-122.42,casual
4,72CC2218DF973489,electric_bike,2021-01-26 09:04:22,2021-01-26 09:11:36,NaN,NaN,NaN,NaN,37.79,-122.41,37.78,-122.39,casual


In [4]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

[HP] Extract ridership data grouped by hour of the day for each day of the month. We only need the start station names and total number of rides. Suggested steps are as follows:

1. Handle the null values in the "start_station_name" and "end_station_name" columns. You may choose to drop the rows if necessary. Provide a short explanation of the choice you made.

2. Extract only the date and hour part of the "started_at" column in a new column, e.g., "2020-12-15,6" for 6:00 am to 6:59 am on 2020-12-15. You may need to use the to_datetime function on the pandas column "started_at".

3. Group the columns by the new column such that the final set of columns are as follows:
*   Date and Hour
*   List of Locations from which rides started in the Date and Hour
*   Total Rides in the Date and Hour
You may need to use the [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function on pandas.

Explain the steps you used to extract the data

In [2]:
## Your code for HP goes here.
import numpy as np

# 1) Handle null station names
# We need "start station names" in the final output, so rows without start_station_name cannot contribute.
# We'll also drop rows with missing end_station_name to keep ride records consistent.
df_hp = df.dropna(subset=["start_station_name", "end_station_name"]).copy()

# 2) Create "date_hour" column from started_at
df_hp["started_at"] = pd.to_datetime(df_hp["started_at"], errors="coerce")

# Drop rows where started_at couldn't be parsed
df_hp = df_hp.dropna(subset=["started_at"]).copy()

# Format: "YYYY-MM-DD,H" (H is 0-23)
df_hp["date_hour"] = (
    df_hp["started_at"].dt.date.astype(str) + "," + df_hp["started_at"].dt.hour.astype(str)
)

# 3) Group by date_hour
grouped = (
    df_hp.groupby("date_hour")
        .agg(
            start_station_locations=("start_station_name", lambda s: sorted(s.unique().tolist())),
            total_rides=("start_station_name", "size")
        )
        .reset_index()
        .rename(columns={"date_hour": "date_and_hour"})
)

grouped.head()

,date_and_hour,start_station_locations,total_rides
0,"2021-01-01,0","[1st St at Folsom St, 20th St at Bryant St, 20...",34
1,"2021-01-01,1","[23rd Ave at Clement St, 27th St at MLK Jr Way...",32
2,"2021-01-01,10","[10th Ave at Irving St, 17th St at Dolores St,...",87
3,"2021-01-01,11","[10th Ave at Irving St, 11th St at Bryant St, ...",115
4,"2021-01-01,12","[10th Ave at Irving St, 13th St at Webster St,...",154


< Your answer for HP goes here >

*   Null handling: I dropped rows with missing
start_station_name and end_station_name. This is appropriate because the task requires a list of start station names, and missing station names cannot be grouped into meaningful station-location lists. Dropping these rows prevents creating invalid or misleading station groupings.

*   Datetime extraction: I converted started_at to pandas datetime using pd.to_datetime. Then I extracted the date and hour components and stored them as a single string key in the format YYYY-MM-DD,H (e.g., 2021-01-15,6) so that all rides occurring within the same hour window (6:00–6:59) are grouped together.

*   Grouping and aggregation: I grouped the dataset by this date_and_hour key. For each group, I computed:

     *   a unique sorted list of start_station_name values to represent the set of ride origins during that hour, and

     *   total_rides as the count of rows in the group (each row = one ride).

In [3]:
output_file = "preprocessed_baywheels_01_2021.csv"
grouped.to_csv(output_file, index=False)
output_file

'preprocessed_baywheels_01_2021.csv'

Save your grouped data in a file called **preprocessed_baywheels_month_year.csv**, where month and year refer to the dataset you are working on based on your SJSU ID. Submit this file along with the ipynb solution file.

# References
Include ALL your references here.
*   Bay Wheels trip data source: https://s3.amazonaws.com/baywheels-data/index.html

*   pandas groupby documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

# What to turn in:
1. The ipynb solution file, which includes the references
2. If you use Colab or GitHub for version control, please share a link to your notebook or GitHub repository
3. The preprocessed file